In [26]:
# --- Install dependencies (run once) ---
# !pip install --upgrade transformers huggingface_hub torch pillow scikit-learn python-dotenv pymupdf faiss-cpu langchain langchain-community

import os
import torch
import numpy as np
import fitz  # PyMuPDF
import io, base64
from PIL import Image
from dotenv import load_dotenv
from transformers import CLIPProcessor, CLIPModel
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage

# --- Load env (not required for Ollama, but kept for compatibility) ---
load_dotenv()

# --- Initialize CLIP model ---
model_id = "openai/clip-vit-base-patch32"
clip_model = CLIPModel.from_pretrained(model_id)
clip_processor = CLIPProcessor.from_pretrained(model_id)
clip_model.eval()
print("✅ CLIP model loaded successfully")

# === Embedding functions ===
def embed_image(image_data):
    if isinstance(image_data, str):
        image = Image.open(image_data).convert("RGB")
    else:
        image = image_data
    inputs = clip_processor(images=image, return_tensors="pt")
    with torch.no_grad():
        features = clip_model.get_image_features(**inputs)
        features = features / features.norm(dim=-1, keepdim=True)
        return features.squeeze().numpy()

def embed_text(text):
    inputs = clip_processor(
        text=text, return_tensors="pt", padding=True, truncation=True, max_length=77
    )
    with torch.no_grad():
        features = clip_model.get_text_features(**inputs)
        features = features / features.norm(dim=-1, keepdim=True)
        return features.squeeze().numpy()

# === Process PDF ===
pdf_path = "multimodal_sample.pdf"   # change to your PDF
doc = fitz.open(pdf_path)

all_docs, all_embeddings, image_data_store = [], [], {}
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

for i, page in enumerate(doc):
    # --- Process Text ---
    text = page.get_text()
    if text.strip():
        temp_doc = Document(page_content=text, metadata={"page": i, "type": "text"})
        text_chunks = splitter.split_documents([temp_doc])
        for chunk in text_chunks:
            embedding = embed_text(chunk.page_content)
            all_embeddings.append(embedding)
            all_docs.append(chunk)

    # --- Process Images ---
    for img_index, img in enumerate(page.get_images(full=True)):
        try:
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            pil_image = Image.open(io.BytesIO(image_bytes)).convert("RGB")

            image_id = f"page_{i}_img_{img_index}"

            # Save as base64
            buffered = io.BytesIO()
            pil_image.save(buffered, format="PNG")
            img_base64 = base64.b64encode(buffered.getvalue()).decode()
            image_data_store[image_id] = img_base64

            embedding = embed_image(pil_image)
            all_embeddings.append(embedding)

            image_doc = Document(
                page_content=f"[Image: {image_id}]",
                metadata={"page": i, "type": "image", "image_id": image_id}
            )
            all_docs.append(image_doc)
        except Exception as e:
            print(f"Error processing image {img_index} on page {i}: {e}")

doc.close()
print(f"✅ Processed {len(all_docs)} docs ({len(all_embeddings)} embeddings)")
print(f"✅ Stored {len(image_data_store)} images")

# === Create FAISS index ===
embeddings_array = np.array(all_embeddings)
vector_store = FAISS.from_embeddings(
    text_embeddings=[(doc.page_content, emb) for doc, emb in zip(all_docs, embeddings_array)],
    embedding=None,
    metadatas=[doc.metadata for doc in all_docs]
)

# === Use Ollama (phi as default, fallback to mistral:q4_0) ===
try:
    llm = ChatOllama(model="phi")
    print("✅ Using Ollama model: phi")
except Exception as e:
    print("⚠️ Could not load phi, trying mistral:q4_0")
    llm = ChatOllama(model="mistral:q4_0")

# === Retrieval ===
def retrieve_multimodal(query, k=5):
    query_embedding = embed_text(query)
    results = vector_store.similarity_search_by_vector(embedding=query_embedding, k=k)
    return results

# === Create multimodal message ===
def create_multimodal_message(query, retrieved_docs):
    content = []
    content.append({"type": "text", "text": f"Question: {query}\n\nContext:\n"})

    text_docs = [doc for doc in retrieved_docs if doc.metadata.get("type") == "text"]
    image_docs = [doc for doc in retrieved_docs if doc.metadata.get("type") == "image"]

    if text_docs:
        text_context = "\n\n".join([
            f"[Page {doc.metadata['page']}]: {doc.page_content}" for doc in text_docs
        ])
        content.append({"type": "text", "text": f"Text excerpts:\n{text_context}\n"})

    for doc in image_docs:
        image_id = doc.metadata.get("image_id")
        if image_id in image_data_store:
            content.append({"type": "text", "text": f"[Image from page {doc.metadata['page']}] included"})

    content.append({"type": "text", "text": "Please answer based on the provided context."})
    return HumanMessage(content=content)

# === Main pipeline ===
def multimodal_pdf_rag_pipeline(query):
    retrieved = retrieve_multimodal(query, k=5)
    message = create_multimodal_message(query, retrieved)
    response = llm.invoke([message])

    print(f"\nRetrieved {len(retrieved)} documents:")
    for doc in retrieved:
        print(f"- {doc.metadata.get('type')} (page {doc.metadata.get('page')})")
    print("\n")
    return response.content

# === Run queries ===
if __name__ == "__main__":
    queries = [
        "What does the chart on page 1 show about revenue trends?",
        "Summarize the main findings from the document",
        "What visual elements are present in the document?"
    ]

    for q in queries:
        print(f"\n❓ Query: {q}")
        print("-" * 50)
        ans = multimodal_pdf_rag_pipeline(q)
        print(f"💡 Answer: {ans}")


✅ CLIP model loaded successfully


`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


✅ Processed 2 docs (2 embeddings)
✅ Stored 1 images
✅ Using Ollama model: phi

❓ Query: What does the chart on page 1 show about revenue trends?
--------------------------------------------------

Retrieved 2 documents:
- text (page 0)
- image (page 0)


💡 Answer:  The chart on page 1 shows that revenue trends show a steady increase in annual revenue across three quarters, with the highest growth recorded in Q3. Q1 showed a moderate increase as new product lines were introduced, and Q2 outperformed Q1 due to marketing campaigns. Q3 had exponential growth due to global expansion.


❓ Query: Summarize the main findings from the document
--------------------------------------------------

Retrieved 2 documents:
- text (page 0)
- image (page 0)


💡 Answer:  The user wants a summary of the annual revenue trends across Q1, Q2, and Q3 from a document. The assistant will summarize that the document shows that the revenue grew steadily with the highest growth recorded in Q3, which had exponenti